Import librairies

In [3]:
import pandas as pd
import numpy as np
import xarray as xr
import tqdm
import atlite
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches

# EirGrid

## Locations of farms

Read capacity of wind and PV farms

In [ ]:
df_capacity_pv = pd.read_csv('../Data_Final/EirGrid/capacity_pv_1724_eir.csv',
                               index_col = 2,
                               parse_dates=True)

In [ ]:
df_capacity_wind = pd.read_csv('../Data_Final/EirGrid/capacity_wind_9224_eir.csv',
                               index_col = 2,
                               parse_dates=True)

Rename the columns to be correctly read by Atlite

In [ ]:
df_capacity_pv = df_capacity_pv.dropna().rename(columns={'Capacity (MW)':'capacity', 'Latitude':'y', 'Longitude':'x'})
df_capacity_pv = df_capacity_pv[df_capacity_pv.index < '2024'] # Make sure to remove all data before 2024

df_capacity_wind = df_capacity_wind.dropna().rename(columns={'Capacity (MW)':'capacity', 'Latitude':'y', 'Longitude':'x'})
df_capacity_wind = df_capacity_wind[df_capacity_wind.index < '2024'] # Make sure to remove all data before 2024

# Atlite

## Create Cutout for Ireland

Define function to create a time dependent layout of the capacity from the dataframe

In [ ]:
def get_time_dependent_capacity_distribution(cutout_ie: atlite.Cutout, df_capacity: pd.DataFrame):
    capacity_layout = cutout_ie.data['roughness'].copy()
    capacity_layout.name = 'Capacity'
    capacity_layout[:,:,:] = 0.

    # Iterate over all capacity installations
    for idx, row in tqdm(df_capacity.reset_index().iterrows(), total= df_capacity.shape[0]):
        cap = row['capacity'] # Capacity values
        df_capacity_i = pd.DataFrame([row])
        layout = cutout_ie.layout_from_capacity_list(df_capacity_i, col="capacity")

        capacity_layout[capacity_layout['time'] >= row['Connection date']] += layout

    return capacity_layout

### Wind

In [ ]:
ds_uv100_to2018 = xr.open_dataset('../Data/ERA5/ERA5_100m_u_v_hourly_2009_2018.nc')
ds_uv100_2019_2023 = xr.open_dataset('../Data/ERA5/ERA5_100m_u_v_hourly_2019_2023.nc')
ds_roughness_2018_2023 = xr.open_dataset('../Data/ERA5/ERA5_forecast_surface_roughness_geopotential_hourly_2018_2023.nc')
ds_roughness_to2017 = xr.open_dataset('../Data/ERA5/ERA5_forecast_surface_roughness_geopotential_hourly_2012_2017.nc')
ds_uv100_to2018 = ds_uv100_to2018.sel(time=slice("2014-01-01", "2018-12-31"))
ds_roughness_to2018 = ds_roughness_to2017.sel(time=slice("2014-01-01", "2017-12-31"))

/home/boris/anaconda3/lib/python3.11/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.24.2
  warnings.warn(


Some variables need to be formated

In [ ]:
ds_uv100_2019_2023 = ds_uv100_2019_2023.reduce(np.nansum, dim='expver',keep_attrs=True)
ds_roughness_2018_2023 = ds_roughness_2018_2023.reduce(np.nansum, dim='expver',keep_attrs=True)

In [ ]:
ds_uv100_2019_2023 = ds_uv100_2019_2023.reduce(np.nansum, dim='expver',keep_attrs=True)
ds_roughness_2018_2023 = ds_roughness_2018_2023.reduce(np.nansum, dim='expver',keep_attrs=True)

ValueError: Dataset does not contain the dimensions: ['expver']

In [ ]:
ds_uv100 = xr.concat([ds_uv100_to2018.rename({'lat':'latitude', 'lon':'longitude'}), ds_uv100_2019_2023], dim='time')
ds_roughness = xr.concat([ds_roughness_to2017, ds_roughness_2018_2023], dim='time')

In [ ]:
ds = xr.merge([ds_uv100, ds_roughness])
ds = ds.sel(time=slice("2014-01-01", "2023-12-31"))

Loads the climate variables in Atlite

This can only be done via the function get_cutout_from_era5_data which is not originally in the Atlite scripts but has been added by the authors

In [ ]:
cutout_ie = atlite.cutout.get_cutout_from_era5_data('path', ds, ['wind'])

### PV